# Library

This chapter contains all the functions that are used in other chapters of the documents. There is the following segregation of functions/classes by topic:
* **Imports** - all the imports and pre-definitions
* **EDA** - the tools for exploration the data nature (i.e. functions for box plot and distribution drawing; NaN and sanity checker, etc) 
* **Feature engineering** - tools for extracting new features out of the data or removing outliers
* **Statistical testing** - tools for testing the hypothesises (i.e. class for t-test or ANOVA)
* **FS interaction** - tools for reading / writing files in FS or merging / separating data structures (i.e. `pd.DataFrame`)

## Imports

In [296]:
# Stat testing
from scipy import stats
from scipy.stats import mannwhitneyu

# Core
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# RE
import re

## EDA tools

This function analyze DataFrame columns to identify and summarize non-numeric values.

In [297]:
def count_non_number_values(df: pd.DataFrame, feature_list: list) -> pd.DataFrame:
    """
    Analyzes specified columns in a pandas DataFrame to detect non-numeric values 
    (i.e., values that are not of type int or float). For each column, the function 
    provides example non-numeric entries, their count, and their relative frequency.

    :param df: A pandas DataFrame containing the dataset.
    :param feature_list: A list of column names (features) to evaluate.
    :return: A pandas DataFrame summarizing the count and frequency of non-numeric values 
             per specified column, along with a few example values.
    """

    # Dictionary to store results for each specified feature/column
    results = {}

    # Loop over each feature in the provided list
    for col in feature_list:
        # Proceed only if the column exists in the DataFrame
        if col in df.columns:
            # Identify values in the column that are NOT integers or floats
            non_int_values = df[col][df[col].apply(lambda x: not isinstance(x, (int, float)))]

            # Store the analysis results for the current column
            results[col] = {
                # Collect up to 5 unique examples of non-numeric values
                "Examples": non_int_values.unique()[:5],

                # Count of non-numeric entries in the column
                "Non-Int Count": non_int_values.shape[0],

                # Relative frequency (%) of non-numeric values in the column
                "Relative frequency": f"{round(non_int_values.shape[0] / df[col].shape[0] * 100, 2)}%",
            }

    # Convert the results dictionary to a DataFrame, using feature names as the index
    return pd.DataFrame.from_dict(results, orient="index")

This function plots boxplots for a user-specified list of numeric features from a DataFrame, validating that all selected columns exist and are of numeric type.

In [298]:
def plot_boxplots(data, numerical_features):
    """
    Plots boxplots for a user-specified list of numeric features from a DataFrame.
    
    :param data: The input DataFrame containing the data to visualize.
    :param numerical_features: List of column names to plot; all must exist in the DataFrame and be numeric.
    :raises ValueError: If any specified feature is missing from the DataFrame or is not of a numeric type.
    """

    # Check if all provided features exist in the DataFrame
    missing = [col for col in numerical_features if col not in data.columns]
    if len(missing) > 0:
        raise ValueError(f"The following features are missing from the DataFrame: {missing}")

    # Check that all selected features are numeric
    non_numeric = [col for col in numerical_features if not np.issubdtype(data[col].dtype, np.number)]
    if non_numeric:
        raise ValueError(f"The following features are not numeric: {non_numeric}")

    # Calculate number of plots (columns) and rows needed (2 plots per row)
    num_cols = len(numerical_features)
    num_rows = (num_cols + 1) // 2

    # Create a grid of subplots with 2 columns per row
    fig, axes = plt.subplots(num_rows, 2, figsize=(20, num_rows * 5))
    axes = axes.flatten()

    # Loop through each numeric feature and plot a boxplot
    for i, col in enumerate(numerical_features):
        ax = axes[i]
        plot_data = data[col]
        ax.set_title(f'Boxplot of {col}')

        sns.boxplot(
            y=plot_data,
            ax=ax,
            color='skyblue',
            width=0.4,
            flierprops={'marker': 'o', 'color': 'red', 'markersize': 5}
        )

        ax.set_ylabel(col)

    # Remove unused axes
    for i in range(num_cols, len(axes)):
        fig.delaxes(axes[i])

    plt.tight_layout()
    plt.show()

This function plots histograms with KDE curves for a specified list of numeric features to visualize their distributions. It helps identify patterns, skewness, and modality across multiple variables in a clean grid layout.

In [299]:
def plot_features_density(data, features):
    """
    Plots histogram and KDE (Kernel Density Estimate) curves for each specified numeric feature in the DataFrame.

    :param data: A pandas DataFrame containing the data.
    :param features: List of column names to plot; all must exist in the DataFrame and be numeric.
    :raises ValueError: If any specified feature is missing or not numeric.
    """

    # Check if all features are present in the DataFrame
    missing = [col for col in features if col not in data.columns]
    if missing:
        raise ValueError(f"The following features are missing from the DataFrame: {missing}")

    # Check that all features are numeric
    non_numeric = [col for col in features if not np.issubdtype(data[col].dtype, np.number)]
    if non_numeric:
        raise ValueError(f"The following features are not numeric: {non_numeric}")

    # Calculate number of plots and rows needed (2 per row)
    num_cols = len(features)
    num_rows = (num_cols + 1) // 2

    # Set overall figure size
    plt.figure(figsize=(15, num_rows * 5))

    # Loop through features and plot histogram + KDE for each
    for i, col in enumerate(features):
        plt.subplot(num_rows, 2, i + 1)

        # Plot histogram (cut off at 95th percentile to avoid long tails)
        plt.hist(
            data[col],
            bins=50,
            range=(data[col].min(), data[col].quantile(0.95)),
            alpha=0.6,
            color='g',
            density=True,
            label='Histogram'
        )

        # Plot KDE curve
        sns.kdeplot(
            data[col],
            color='r',
            linewidth=2,
            label='KDE Trend'
        )

        plt.xlabel(col)
        plt.ylabel('Density')
        plt.title(f'Distribution of {col}')
        plt.legend()

    # Adjust layout to avoid overlap
    plt.tight_layout()
    plt.show()

## Feature engineering tools

This function checks if a message contains any personal words from a predefined set.

In [300]:
def contains_personal_words(text: str, personal_words: set[str]):
    """
    Determines whether the input text contains any personal words
    from the predefined set PERSONAL_WORDS.

    :param personal_words: set of personal words
    :param text: The input string (e.g., a message or comment).
    :return: True if at least one personal word is present, False otherwise.
    """
    words = set(str(text).lower().split())
    return not personal_words.isdisjoint(words)

This function creates a boolean feature indicating whether the message contains a mention of the given account.

In [301]:
def extract_mention_match(message: str, account: str) -> bool:
    """
    Checks if the message contains any @mention that does NOT match the given account.

    :param message: Text message that may contain @mentions.
    :param account: Target account name to match against mentions.

    :returns True if any mention matches the account name (case-insensitive), False otherwise.
    """
    # Extract all words that follow the @ symbol (e.g., @username)
    mentions = re.findall(r'@(\w+)', message)

    # Compare each mention to the account (case-insensitive)
    return any(m.lower() != account.lower() for m in mentions)

## Statistical testing tools

Following class functionality:
1. This class performs statistical hypothesis testing between two numerical samples.
2. It automatically chooses between a **T-test** and **Mann-Whitney U test** based on normality and variance checks.

In [302]:
class TwoSampleStatisticalTests:
    def __init__(self, first_dataset: pd.Series, second_dataset: pd.Series, significance_level: float):
        """
        Initialize the class with two datasets and a significance level.

        :param first_dataset: First sample (pandas Series)
        :param second_dataset: Second sample (pandas Series)
        :param significance_level: Alpha threshold for hypothesis testing (e.g., 0.05)
        """
        self._first_dataset = first_dataset
        self._second_dataset = second_dataset
        self._significance_level = significance_level

    def _check_normality(self) -> bool:
        """
        Test for normal distribution using Shapiro-Wilk’s test.

        H0: The data is normally distributed  
        Ha: The data is not normally distributed

        :return: True if both datasets are normally distributed, else False
        """
        _, pvalue_first = stats.shapiro(self._first_dataset)
        _, pvalue_second = stats.shapiro(self._second_dataset)

        print(f"The result of the p-value when checking the normality for the first dataset: {pvalue_first}")
        print(f"The result of the p-value when checking the normality for the second dataset: {pvalue_second}")

        return pvalue_first >= self._significance_level and pvalue_second >= self._significance_level

    def _check_variance_homogeneity(self) -> bool:
        """
        Test for homogeneity of variances using Levene's test.

        H0: Variances are equal (homogeneous)  
        Ha: Variances are different

        :return: True if variances are equal, else False
        """
        _, pvalue = stats.levene(self._first_dataset, self._second_dataset)

        print(f"The result of the p-value when checking the variance uniform: {pvalue}")

        return pvalue >= self._significance_level

    def _t_test(self, is_one_tailed: bool = False) -> float:
        """
        Perform an independent two-sample T-test assuming equal variances.

        :param is_one_tailed: If True, calculates one-tailed p-value.
        :return: p-value from T-test
        """
        t_stat, p_value = stats.ttest_ind(self._first_dataset, self._second_dataset, equal_var=True)

        # Adjust for one-tailed test if requested
        return p_value if not is_one_tailed else (p_value / 2 if t_stat > 0 else 1 - (p_value / 2))

    def _mann_whitney_test(self, is_one_tailed: bool = False) -> float:
        """
        Perform the Mann-Whitney U test (non-parametric alternative to T-test).

        :param is_one_tailed: If True, uses 'greater' alternative.
        :return: p-value from Mann-Whitney test
        """
        alternative = 'greater' if is_one_tailed else 'two-sided'
        _, p_value = mannwhitneyu(self._first_dataset, self._second_dataset, alternative=alternative)
        return p_value

    def test_two_numerical_samples(self, is_one_tailed: bool = False):
        """
        Runs the appropriate hypothesis test between two numerical samples.

        - If both samples are normally distributed AND variances are equal: T-test is used  
        - Otherwise: Mann-Whitney U test is used

        Prints the test used, the final p-value, and the hypothesis decision.
        """
        # Step 1: Check distribution
        is_normally_distributed = self._check_normality()

        # Step 2: Check variance homogeneity
        is_variance_homogeneous = self._check_variance_homogeneity()

        # Step 3: Choose test based on assumptions
        if is_normally_distributed and is_variance_homogeneous:
            print("T-test was chosen")
            p_value = self._t_test(is_one_tailed)
        else:
            print("Mann-Whitney was chosen")
            p_value = self._mann_whitney_test(is_one_tailed)

        # Step 4: Report results
        print(f"Final p-value: {p_value}")

        if p_value <= self._significance_level:
            print("H0 has been rejected, Ha has been accepted")
        else:
            print("H0 was not rejected")

## Dataframe interaction tools

This function standardizes DataFrame column names by converting them to lowercase and replacing spaces and hyphens with underscores, making them easier to work with in code

In [303]:
def clean_column_names(df):
    """
    Converts column names to lowercase and replaces spaces and hyphens with underscores.

    :param df: A pandas DataFrame with original column names.
    :return: DataFrame with cleaned column names.
    """
    df = df.copy()
    df.columns = (
        df.columns
        .str.lower()  # Convert all column names to lowercase
        .str.replace(' ', '_', regex=False)  # Replace spaces with underscores
        .str.replace('-', '_', regex=False)  # Replace hyphens with underscores
        .str.replace(r',|&', '', regex=True)  # Replace commas and ampersands
        .str.replace(r'_+', '_', regex=True)  # Replace duplicates
    )
    return df

Reading dataset from **xlsx**

In [304]:
def read_dataset(brand_name: str, folder: str):
    df = pd.read_excel(f'./data/{folder}/{brand_name}.xlsx')
    return df

# Reading datasets

Defining the folders names

In [305]:
INTERNATIONAL_TYPE = "international"
LOCAL_TYPE = "local"

Defining the datasets names for **international** and **local** companies

In [306]:
INTERNATIONAL_COMPANIES = ['@corendon.nl', '@vakantiediscounter', '@prijsvrijvakanties', '@sunweb', '@tuinederland',
                           '@dreizenvakanties', '@elizawashere']

LOCAL_COMPANIES = ['@accessibletravelnetherlands', '@cherrytraveltours', '@hollandtravel', '@lvtravelagency',
                   '@tomstraveltours', '@tenzing_travel', '@cruisetravelnl']

Read **local** datasets into list of pd.DataFrame

In [307]:
local_companies_dfs = []

for brand_name in LOCAL_COMPANIES:
    current_df = read_dataset(brand_name, LOCAL_TYPE)
    current_df['account'] = brand_name
    current_df['type'] = LOCAL_TYPE

    local_companies_dfs.append(current_df)

local_companies_dfs[0]

,Date,Message,Profile,Network,Engagement,Post interaction rate,Number of comments,"Reactions, Comments & Shares",Number of Likes,Message-ID,Profile-ID,Link,account,type
0,2025-03-01 09:04:43,Discover Nijmegen: The Oldest City in the Neth...,Accessible Travel Netherlands,INSTAGRAM,0.008539,0.008539,0,9,9,18052737773131838,43939331,https://www.instagram.com/p/DGpqqf8o9yE/,@accessibletravelnetherlands,local
1,2025-02-05 11:14:13,Amsterdam should be at the top of your travel ...,Accessible Travel Netherlands,INSTAGRAM,-,-,1,26,25,18101839216492258,43939331,https://www.instagram.com/p/DFsGZ3voYrG/,@accessibletravelnetherlands,local
2,2025-01-31 15:32:04,The Veluwe ❤️\n\nExplore one of the most stunn...,Accessible Travel Netherlands,INSTAGRAM,-,-,0,5,5,18090951523529057,43939331,https://www.instagram.com/p/DFfr8IVoPZf/,@accessibletravelnetherlands,local
3,2025-01-29 16:31:20,Experience the Traditional Gouda Cheese Market...,Accessible Travel Netherlands,INSTAGRAM,-,-,0,9,9,18060269164931558,43939331,https://www.instagram.com/p/DFapIbBoaj7/,@accessibletravelnetherlands,local
4,2025-01-26 16:20:51,"At Accessible Travel Netherlands, we offer 10 ...",Accessible Travel Netherlands,INSTAGRAM,-,-,0,5,5,18051201320326350,43939331,https://www.instagram.com/p/DFS5jAbIHIC/,@accessibletravelnetherlands,local
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,2013-02-19 16:40:03,#deventer #old #historic #city feels like #eng...,Accessible Travel Netherlands,INSTAGRAM,-,-,0,3,3,17842589947035332,43939331,https://www.instagram.com/p/V62rePw7Ge/,@accessibletravelnetherlands,local
203,2013-01-28 13:31:25,At the #oldest #neigbourhood of #panama #city,Accessible Travel Netherlands,INSTAGRAM,-,-,0,6,6,17842581193035332,43939331,https://www.instagram.com/p/VB3mrKw7J4/,@accessibletravelnetherlands,local
204,2012-12-16 20:38:32,"Ons nieuwe huis op Senidup, Archipela de San B...",Accessible Travel Netherlands,INSTAGRAM,-,-,0,2,2,17842564546035332,43939331,https://www.instagram.com/p/TT6TXvQ7PQ/,@accessibletravelnetherlands,local
205,2012-07-18 23:15:16,NaN,Accessible Travel Netherlands,INSTAGRAM,-,-,0,0,0,17842437124035332,43939331,https://www.instagram.com/p/NPYNL_w7LO/,@accessibletravelnetherlands,local


Join the datasets together into one **Dataframe** and save it

In [308]:
local_df = pd.concat(local_companies_dfs, axis=0)
local_df.to_excel(f'./data/{LOCAL_TYPE}.xlsx', index=False)

local_df

,Date,Message,Profile,Network,Engagement,Post interaction rate,Number of comments,"Reactions, Comments & Shares",Number of Likes,Message-ID,Profile-ID,Link,account,type
0,2025-03-01 09:04:43,Discover Nijmegen: The Oldest City in the Neth...,Accessible Travel Netherlands,INSTAGRAM,0.008539,0.008539,0,9,9,18052737773131838,43939331,https://www.instagram.com/p/DGpqqf8o9yE/,@accessibletravelnetherlands,local
1,2025-02-05 11:14:13,Amsterdam should be at the top of your travel ...,Accessible Travel Netherlands,INSTAGRAM,-,-,1,26,25,18101839216492258,43939331,https://www.instagram.com/p/DFsGZ3voYrG/,@accessibletravelnetherlands,local
2,2025-01-31 15:32:04,The Veluwe ❤️\n\nExplore one of the most stunn...,Accessible Travel Netherlands,INSTAGRAM,-,-,0,5,5,18090951523529057,43939331,https://www.instagram.com/p/DFfr8IVoPZf/,@accessibletravelnetherlands,local
3,2025-01-29 16:31:20,Experience the Traditional Gouda Cheese Market...,Accessible Travel Netherlands,INSTAGRAM,-,-,0,9,9,18060269164931558,43939331,https://www.instagram.com/p/DFapIbBoaj7/,@accessibletravelnetherlands,local
4,2025-01-26 16:20:51,"At Accessible Travel Netherlands, we offer 10 ...",Accessible Travel Netherlands,INSTAGRAM,-,-,0,5,5,18051201320326350,43939331,https://www.instagram.com/p/DFS5jAbIHIC/,@accessibletravelnetherlands,local
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,2019-08-23 09:39:08,Heeft u mooie foto’s van uw eigen cruise gemaa...,Cruise Travel,INSTAGRAM,-,-,0,22,22,17868232276456526,5621753329,https://www.instagram.com/p/B1gHPoyCygq/,@cruisetravelnl,local
180,2019-07-15 14:54:46,Onze ambassadeur Henk van der Noort heeft een ...,Cruise Travel,INSTAGRAM,-,-,2,26,24,17847898231507730,5621753329,https://www.instagram.com/p/Bz8QXReChhs/,@cruisetravelnl,local
181,2019-07-01 10:31:44,Onze ambassadeur A. Prevo ging aan boord van d...,Cruise Travel,INSTAGRAM,-,-,0,23,23,18079004770055863,5621753329,https://www.instagram.com/p/BzXvIioiGI2/,@cruisetravelnl,local
182,2019-06-20 08:12:23,Verschillende reisbureaus op avontuur naar Ala...,Cruise Travel,INSTAGRAM,-,-,3,49,46,18049625401089614,5621753329,https://www.instagram.com/p/By7KcTpiXN4/,@cruisetravelnl,local


Read **international** datasets into list of pd.DataFrame

In [309]:
international_companies_dfs = []

for brand_name in INTERNATIONAL_COMPANIES:
    current_df = read_dataset(brand_name, INTERNATIONAL_TYPE)
    current_df['account'] = brand_name
    current_df['type'] = INTERNATIONAL_TYPE

    international_companies_dfs.append(current_df)

international_companies_dfs[0]

,Date,Message,Profile,Network,Engagement,Post interaction rate,Number of comments,"Reactions, Comments & Shares",Number of Likes,Message-ID,Profile-ID,Link,account,type
0,2025-03-17 19:01:17,??Ontvang nu extra vroegboekkorting tot €200 é...,Corendon,INSTAGRAM,0.001174,0.001174,3,185,182,18038388467208858,1466583220,https://www.instagram.com/reel/DHT7mD7PW0A/,@corendon.nl,international
1,2025-03-16 14:00:14,"Zon, zee en eindeloze siesta’s… ????☀️ In Span...",Corendon,INSTAGRAM,0.000584,0.000584,0,92,92,18039075278448488,1466583220,https://www.instagram.com/p/DHQ0Zedsckb/,@corendon.nl,international
2,2025-03-16 09:00:39,"☀️ Welkom bij Voyage Belek Golf & Spa, waar ee...",Corendon,INSTAGRAM,0.000838,0.000838,1,132,131,18085134403543445,1466583220,https://www.instagram.com/reel/DHQSGJMB3Nl/,@corendon.nl,international
3,2025-03-15 09:00:32,Hotel Diamond Excellence Resort & Spa in Side ...,Corendon,INSTAGRAM,0.001041,0.001041,0,164,164,17842557165447487,1466583220,https://www.instagram.com/p/DHNtTrRsRIj/,@corendon.nl,international
4,2025-03-14 13:01:48,Last-Minute Deal: €70 Korting op het Ladies of...,Corendon,INSTAGRAM,0.001612,0.001612,5,254,249,18046848881361680,1466583220,https://www.instagram.com/p/DHLkIA1sQxy/,@corendon.nl,international
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2545,2014-09-29 06:40:40,Morning! Zin in een #Grieks ontbijtje. Vlieg t...,Corendon Vliegvakanties,INSTAGRAM,-,-,7,111,104,820010076175750530_1466583220,1466583220,https://www.instagram.com/p/thQtHWslWC/,@corendon.nl,international
2546,2014-09-11 12:23:14,Nu online op www.girlslove2travel.com het vers...,Corendon,INSTAGRAM,-,-,1,77,76,17841752296023221,1466583220,https://www.instagram.com/p/szhmZnMlUM/,@corendon.nl,international
2547,2014-09-11 12:23:14,Nu online op www.girlslove2travel.com het vers...,Corendon Vliegvakanties,INSTAGRAM,-,-,1,77,76,807136532605588748_1466583220,1466583220,https://www.instagram.com/p/szhmZnMlUM/,@corendon.nl,international
2548,2014-09-10 11:18:41,This is truly one of the nicest beaches around...,Corendon,INSTAGRAM,-,-,9,111,102,17841752305023221,1466583220,https://www.instagram.com/p/sw1atpMlV4/,@corendon.nl,international


Join the datasets together into one **Dataframe** and save it

In [310]:
international_df = pd.concat(international_companies_dfs, axis=0)
international_df.to_excel(f'./data/{INTERNATIONAL_TYPE}.xlsx', index=False)

international_df

,Date,Message,Profile,Network,Engagement,Post interaction rate,Number of comments,"Reactions, Comments & Shares",Number of Likes,Message-ID,Profile-ID,Link,account,type
0,2025-03-17 19:01:17,??Ontvang nu extra vroegboekkorting tot €200 é...,Corendon,INSTAGRAM,0.001174,0.001174,3,185,182,18038388467208858,1466583220,https://www.instagram.com/reel/DHT7mD7PW0A/,@corendon.nl,international
1,2025-03-16 14:00:14,"Zon, zee en eindeloze siesta’s… ????☀️ In Span...",Corendon,INSTAGRAM,0.000584,0.000584,0,92,92,18039075278448488,1466583220,https://www.instagram.com/p/DHQ0Zedsckb/,@corendon.nl,international
2,2025-03-16 09:00:39,"☀️ Welkom bij Voyage Belek Golf & Spa, waar ee...",Corendon,INSTAGRAM,0.000838,0.000838,1,132,131,18085134403543445,1466583220,https://www.instagram.com/reel/DHQSGJMB3Nl/,@corendon.nl,international
3,2025-03-15 09:00:32,Hotel Diamond Excellence Resort & Spa in Side ...,Corendon,INSTAGRAM,0.001041,0.001041,0,164,164,17842557165447487,1466583220,https://www.instagram.com/p/DHNtTrRsRIj/,@corendon.nl,international
4,2025-03-14 13:01:48,Last-Minute Deal: €70 Korting op het Ladies of...,Corendon,INSTAGRAM,0.001612,0.001612,5,254,249,18046848881361680,1466583220,https://www.instagram.com/p/DHLkIA1sQxy/,@corendon.nl,international
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1626,2014-08-05 09:33:06,Beleef jij deze zomer een ultiem #elizamomentj...,Eliza was here,INSTAGRAM,-,-,5,20,15,17842862974034381,1258146380,https://www.instagram.com/p/rT8t-nlZRg/,@elizawashere,international
1627,2014-08-04 16:54:05,Lig jij deze zomer hier? Je hebt nog tot morge...,Eliza was here,INSTAGRAM,-,-,4,19,15,17841991168034381,1258146380,https://www.instagram.com/p/rSKY5wFZcT/,@elizawashere,international
1628,2014-07-30 17:08:00,Puur genieten op een ligbedje bij het zwembad....,Eliza was here,INSTAGRAM,-,-,1,16,15,17842038979034381,1258146380,https://www.instagram.com/p/rFUAgoFZV-/,@elizawashere,international
1629,2014-07-29 16:28:38,"Ken je dat gevoel? Je haren in de wind, je voe...",Eliza was here,INSTAGRAM,-,-,9,22,13,17841828277034381,1258146380,https://www.instagram.com/p/rCqtQHlZRN/,@elizawashere,international


Join all the datasets and save **final** dataset

In [311]:
final_df = pd.concat([international_df, local_df], axis=0)
final_df.to_excel("./final.xlsx", index=False)

final_df

,Date,Message,Profile,Network,Engagement,Post interaction rate,Number of comments,"Reactions, Comments & Shares",Number of Likes,Message-ID,Profile-ID,Link,account,type
0,2025-03-17 19:01:17,??Ontvang nu extra vroegboekkorting tot €200 é...,Corendon,INSTAGRAM,0.001174,0.001174,3,185,182,18038388467208858,1466583220,https://www.instagram.com/reel/DHT7mD7PW0A/,@corendon.nl,international
1,2025-03-16 14:00:14,"Zon, zee en eindeloze siesta’s… ????☀️ In Span...",Corendon,INSTAGRAM,0.000584,0.000584,0,92,92,18039075278448488,1466583220,https://www.instagram.com/p/DHQ0Zedsckb/,@corendon.nl,international
2,2025-03-16 09:00:39,"☀️ Welkom bij Voyage Belek Golf & Spa, waar ee...",Corendon,INSTAGRAM,0.000838,0.000838,1,132,131,18085134403543445,1466583220,https://www.instagram.com/reel/DHQSGJMB3Nl/,@corendon.nl,international
3,2025-03-15 09:00:32,Hotel Diamond Excellence Resort & Spa in Side ...,Corendon,INSTAGRAM,0.001041,0.001041,0,164,164,17842557165447487,1466583220,https://www.instagram.com/p/DHNtTrRsRIj/,@corendon.nl,international
4,2025-03-14 13:01:48,Last-Minute Deal: €70 Korting op het Ladies of...,Corendon,INSTAGRAM,0.001612,0.001612,5,254,249,18046848881361680,1466583220,https://www.instagram.com/p/DHLkIA1sQxy/,@corendon.nl,international
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,2019-08-23 09:39:08,Heeft u mooie foto’s van uw eigen cruise gemaa...,Cruise Travel,INSTAGRAM,-,-,0,22,22,17868232276456526,5621753329,https://www.instagram.com/p/B1gHPoyCygq/,@cruisetravelnl,local
180,2019-07-15 14:54:46,Onze ambassadeur Henk van der Noort heeft een ...,Cruise Travel,INSTAGRAM,-,-,2,26,24,17847898231507730,5621753329,https://www.instagram.com/p/Bz8QXReChhs/,@cruisetravelnl,local
181,2019-07-01 10:31:44,Onze ambassadeur A. Prevo ging aan boord van d...,Cruise Travel,INSTAGRAM,-,-,0,23,23,18079004770055863,5621753329,https://www.instagram.com/p/BzXvIioiGI2/,@cruisetravelnl,local
182,2019-06-20 08:12:23,Verschillende reisbureaus op avontuur naar Ala...,Cruise Travel,INSTAGRAM,-,-,3,49,46,18049625401089614,5621753329,https://www.instagram.com/p/By7KcTpiXN4/,@cruisetravelnl,local


# Dataset preparation

## Columns unification

Standardize all column values to a single format (currently only **final_df** used, but in the end it will be split into several groups again)

In [312]:
unified_final_df = clean_column_names(final_df)
unified_final_df.columns

Index(['date', 'message', 'profile', 'network', 'engagement',
       'post_interaction_rate', 'number_of_comments',
       'reactions_comments_shares', 'number_of_likes', 'message_id',
       'profile_id', 'link', 'account', 'type'],
      dtype='object')

## Absence analysis

At the very beginning, it's necessary to check whether **Nan-values** exist in the dataset or not.

As it possible to see there is only **86** absent values in `message` column, which is **0.46%** of all rows in dataset 

In [313]:
missing_values = unified_final_df.isnull().sum()
missing_values

date                          0
message                      86
profile                       0
network                       0
engagement                    0
post_interaction_rate         0
number_of_comments            0
reactions_comments_shares     0
number_of_likes               0
message_id                    0
profile_id                    0
link                          2
account                       0
type                          0
dtype: int64

In [314]:
relative_missing_values = missing_values / len(unified_final_df) * 100
relative_missing_values = relative_missing_values.apply(lambda x: f"{round(x, 2)}%")

relative_missing_values

date                          0.0%
message                      0.41%
profile                       0.0%
network                       0.0%
engagement                    0.0%
post_interaction_rate         0.0%
number_of_comments            0.0%
reactions_comments_shares     0.0%
number_of_likes               0.0%
message_id                    0.0%
profile_id                    0.0%
link                         0.01%
account                       0.0%
type                          0.0%
dtype: object

Additionally, the data types are checked and their compliance with the expected.

In [315]:
unified_final_df.dtypes

date                         datetime64[ns]
message                              object
profile                              object
network                              object
engagement                           object
post_interaction_rate                object
number_of_comments                    int64
reactions_comments_shares             int64
number_of_likes                      object
message_id                           object
profile_id                            int64
link                                 object
account                              object
type                                 object
dtype: object

**Potential collisions**:
- `number_of_likes` - expected to be **int64**, but it's an **object** 
- `engagement` - expected to be **float64**, but it's an **object** 
- `post_interaction_rate` -  expected to be **float64**, but it's an **object** 

All the collision are potential candidates for **absence analysis** == we have to check do we have enough data to fill / drop missing lines and continue the analysis

In [316]:
POTENTIAL_COLLISIONS = ['number_of_likes', 'post_interaction_rate', 'engagement']

The **Engagement** and **Post interaction rate** were initially proposed as potential target metrics, but it is necessary to check whether we have enough data (not only target metrics are checked, but also all potential candidates for incorrect data)

In [317]:
final_non_number = count_non_number_values(unified_final_df, POTENTIAL_COLLISIONS)
final_non_number

,Examples,Non-Int Count,Relative frequency
number_of_likes,[-],95,0.45%
post_interaction_rate,[-],8647,40.88%
engagement,[-],8647,40.88%


As we can observe, we cannot simply drop the rows where `engagement` or `post_interaction_rate` are equal to **"-"**, as this would violate the requirement of maintaining at least **20,000 rows** in the dataset. 

Replacing these values with **the mean** is also not viable, since nearly half of the target metric would become unreliable or distorted. 

Therefore, it is necessary to consider **an alternative target metric** that ensures data integrity while preserving the sample size.

## Data and features selection

### Absent data dropping

Since rows with **Nan-values** make up only **0.41%** of the total, it's possible just to drop them without breaking any requirements

In [318]:
non_nan_final_df = unified_final_df.dropna()
non_nan_final_df

,date,message,profile,network,engagement,post_interaction_rate,number_of_comments,reactions_comments_shares,number_of_likes,message_id,profile_id,link,account,type
0,2025-03-17 19:01:17,??Ontvang nu extra vroegboekkorting tot €200 é...,Corendon,INSTAGRAM,0.001174,0.001174,3,185,182,18038388467208858,1466583220,https://www.instagram.com/reel/DHT7mD7PW0A/,@corendon.nl,international
1,2025-03-16 14:00:14,"Zon, zee en eindeloze siesta’s… ????☀️ In Span...",Corendon,INSTAGRAM,0.000584,0.000584,0,92,92,18039075278448488,1466583220,https://www.instagram.com/p/DHQ0Zedsckb/,@corendon.nl,international
2,2025-03-16 09:00:39,"☀️ Welkom bij Voyage Belek Golf & Spa, waar ee...",Corendon,INSTAGRAM,0.000838,0.000838,1,132,131,18085134403543445,1466583220,https://www.instagram.com/reel/DHQSGJMB3Nl/,@corendon.nl,international
3,2025-03-15 09:00:32,Hotel Diamond Excellence Resort & Spa in Side ...,Corendon,INSTAGRAM,0.001041,0.001041,0,164,164,17842557165447487,1466583220,https://www.instagram.com/p/DHNtTrRsRIj/,@corendon.nl,international
4,2025-03-14 13:01:48,Last-Minute Deal: €70 Korting op het Ladies of...,Corendon,INSTAGRAM,0.001612,0.001612,5,254,249,18046848881361680,1466583220,https://www.instagram.com/p/DHLkIA1sQxy/,@corendon.nl,international
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,2019-08-23 09:39:08,Heeft u mooie foto’s van uw eigen cruise gemaa...,Cruise Travel,INSTAGRAM,-,-,0,22,22,17868232276456526,5621753329,https://www.instagram.com/p/B1gHPoyCygq/,@cruisetravelnl,local
180,2019-07-15 14:54:46,Onze ambassadeur Henk van der Noort heeft een ...,Cruise Travel,INSTAGRAM,-,-,2,26,24,17847898231507730,5621753329,https://www.instagram.com/p/Bz8QXReChhs/,@cruisetravelnl,local
181,2019-07-01 10:31:44,Onze ambassadeur A. Prevo ging aan boord van d...,Cruise Travel,INSTAGRAM,-,-,0,23,23,18079004770055863,5621753329,https://www.instagram.com/p/BzXvIioiGI2/,@cruisetravelnl,local
182,2019-06-20 08:12:23,Verschillende reisbureaus op avontuur naar Ala...,Cruise Travel,INSTAGRAM,-,-,3,49,46,18049625401089614,5621753329,https://www.instagram.com/p/By7KcTpiXN4/,@cruisetravelnl,local


In [319]:
non_nan_final_df.isnull().sum()

date                         0
message                      0
profile                      0
network                      0
engagement                   0
post_interaction_rate        0
number_of_comments           0
reactions_comments_shares    0
number_of_likes              0
message_id                   0
profile_id                   0
link                         0
account                      0
type                         0
dtype: int64

Compared to other features, `number_of_lines` shows only **0.45%** of incorrect values, so we can simply drop the incorrect rows without breaking the requirements.

In [320]:
dropped_final_df = non_nan_final_df[non_nan_final_df['number_of_likes'].apply(lambda x: isinstance(x, int))]
dropped_final_df

,date,message,profile,network,engagement,post_interaction_rate,number_of_comments,reactions_comments_shares,number_of_likes,message_id,profile_id,link,account,type
0,2025-03-17 19:01:17,??Ontvang nu extra vroegboekkorting tot €200 é...,Corendon,INSTAGRAM,0.001174,0.001174,3,185,182,18038388467208858,1466583220,https://www.instagram.com/reel/DHT7mD7PW0A/,@corendon.nl,international
1,2025-03-16 14:00:14,"Zon, zee en eindeloze siesta’s… ????☀️ In Span...",Corendon,INSTAGRAM,0.000584,0.000584,0,92,92,18039075278448488,1466583220,https://www.instagram.com/p/DHQ0Zedsckb/,@corendon.nl,international
2,2025-03-16 09:00:39,"☀️ Welkom bij Voyage Belek Golf & Spa, waar ee...",Corendon,INSTAGRAM,0.000838,0.000838,1,132,131,18085134403543445,1466583220,https://www.instagram.com/reel/DHQSGJMB3Nl/,@corendon.nl,international
3,2025-03-15 09:00:32,Hotel Diamond Excellence Resort & Spa in Side ...,Corendon,INSTAGRAM,0.001041,0.001041,0,164,164,17842557165447487,1466583220,https://www.instagram.com/p/DHNtTrRsRIj/,@corendon.nl,international
4,2025-03-14 13:01:48,Last-Minute Deal: €70 Korting op het Ladies of...,Corendon,INSTAGRAM,0.001612,0.001612,5,254,249,18046848881361680,1466583220,https://www.instagram.com/p/DHLkIA1sQxy/,@corendon.nl,international
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,2019-08-23 09:39:08,Heeft u mooie foto’s van uw eigen cruise gemaa...,Cruise Travel,INSTAGRAM,-,-,0,22,22,17868232276456526,5621753329,https://www.instagram.com/p/B1gHPoyCygq/,@cruisetravelnl,local
180,2019-07-15 14:54:46,Onze ambassadeur Henk van der Noort heeft een ...,Cruise Travel,INSTAGRAM,-,-,2,26,24,17847898231507730,5621753329,https://www.instagram.com/p/Bz8QXReChhs/,@cruisetravelnl,local
181,2019-07-01 10:31:44,Onze ambassadeur A. Prevo ging aan boord van d...,Cruise Travel,INSTAGRAM,-,-,0,23,23,18079004770055863,5621753329,https://www.instagram.com/p/BzXvIioiGI2/,@cruisetravelnl,local
182,2019-06-20 08:12:23,Verschillende reisbureaus op avontuur naar Ala...,Cruise Travel,INSTAGRAM,-,-,3,49,46,18049625401089614,5621753329,https://www.instagram.com/p/By7KcTpiXN4/,@cruisetravelnl,local


In [321]:
count_non_number_values(dropped_final_df, ['number_of_likes'])

,Examples,Non-Int Count,Relative frequency
number_of_likes,[],0,0.0%


Final amount of the rows is **20969** that fulfills the requirement of **20000** rows in the dataset

### Feature selection

To define the potential candidates for feature dropping it's necessary to check unique values of each feature

In [322]:
unique_values = {
    "Feature": [],
    "Unique Values": []
}

for col in dropped_final_df.columns:
    unique_values["Feature"].append(col)
    unique_values["Unique Values"].append(dropped_final_df[col].unique().tolist())
    
pd.DataFrame(unique_values)

,Feature,Unique Values
0,date,"[2025-03-17 19:01:17, 2025-03-16 14:00:14, 202..."
1,message,[??Ontvang nu extra vroegboekkorting tot €200 ...
2,profile,"[Corendon, Corendon., Corendon Vliegvakanties,..."
3,network,[INSTAGRAM]
4,engagement,"[0.0011743198466401758, 0.000583900838405443, ..."
5,post_interaction_rate,"[0.0011743198466401758, 0.000583900838405443, ..."
6,number_of_comments,"[3, 0, 1, 5, 129, 17, 6, 2, 16, 28, 13, 10, 9,..."
7,reactions_comments_shares,"[185, 92, 132, 164, 254, 906, 826, 138, 236, 1..."
8,number_of_likes,"[182, 92, 131, 164, 249, 777, 809, 132, 231, 1..."
9,message_id,"[18038388467208858, 18039075278448488, 1808513..."


For the further analysis it's not necessary to hold all the features. Here are the candidates to drop:
- `profile` - duplicate the feature `account`, but contains collisions like "Corendon" and "Corendon."
- `profile_id` - equal to feature `account`
- `network` - contains only 1 value
- `engagement` - won't be used for **target feature** (reasons above) - `number_of_likes` and `number_of_comments` will be used instead
- `post_interaction_rate` - won't be used for **target feature** (reasons above) - `number_of_likes` and `number_of_comments` will be used instead
- `reactions_comments_shares` - in the documentation of **FanPage Karma** it's said that this field is summarization of **likes**, **comments** and **shares**. Since shares won't be used for target, the field is **useless**

In [323]:
DROP_CANDIDATES = [
    'profile',
    'profile_id',
    'network',
    'engagement',
    'post_interaction_rate',
    'reactions_comments_shares',
]

Drop all the useless features

In [324]:
feature_dropped_final_df = dropped_final_df.drop(DROP_CANDIDATES, axis=1)
feature_dropped_final_df

,date,message,number_of_comments,number_of_likes,message_id,link,account,type
0,2025-03-17 19:01:17,??Ontvang nu extra vroegboekkorting tot €200 é...,3,182,18038388467208858,https://www.instagram.com/reel/DHT7mD7PW0A/,@corendon.nl,international
1,2025-03-16 14:00:14,"Zon, zee en eindeloze siesta’s… ????☀️ In Span...",0,92,18039075278448488,https://www.instagram.com/p/DHQ0Zedsckb/,@corendon.nl,international
2,2025-03-16 09:00:39,"☀️ Welkom bij Voyage Belek Golf & Spa, waar ee...",1,131,18085134403543445,https://www.instagram.com/reel/DHQSGJMB3Nl/,@corendon.nl,international
3,2025-03-15 09:00:32,Hotel Diamond Excellence Resort & Spa in Side ...,0,164,17842557165447487,https://www.instagram.com/p/DHNtTrRsRIj/,@corendon.nl,international
4,2025-03-14 13:01:48,Last-Minute Deal: €70 Korting op het Ladies of...,5,249,18046848881361680,https://www.instagram.com/p/DHLkIA1sQxy/,@corendon.nl,international
...,...,...,...,...,...,...,...,...
179,2019-08-23 09:39:08,Heeft u mooie foto’s van uw eigen cruise gemaa...,0,22,17868232276456526,https://www.instagram.com/p/B1gHPoyCygq/,@cruisetravelnl,local
180,2019-07-15 14:54:46,Onze ambassadeur Henk van der Noort heeft een ...,2,24,17847898231507730,https://www.instagram.com/p/Bz8QXReChhs/,@cruisetravelnl,local
181,2019-07-01 10:31:44,Onze ambassadeur A. Prevo ging aan boord van d...,0,23,18079004770055863,https://www.instagram.com/p/BzXvIioiGI2/,@cruisetravelnl,local
182,2019-06-20 08:12:23,Verschillende reisbureaus op avontuur naar Ala...,3,46,18049625401089614,https://www.instagram.com/p/By7KcTpiXN4/,@cruisetravelnl,local


`message_id` is a potential candidate for **index**, but for approval it's necessary to check whether it has duplicates or not

In [325]:
feature_dropped_final_df['message_id'].duplicated().any()

False

Hence `message_id` doesn't have duplicates, it'll be used as an index

In [326]:
selected_final_df = feature_dropped_final_df.set_index('message_id')
selected_final_df

,date,message,number_of_comments,number_of_likes,link,account,type
message_id,,,,,,,
18038388467208858,2025-03-17 19:01:17,??Ontvang nu extra vroegboekkorting tot €200 é...,3,182,https://www.instagram.com/reel/DHT7mD7PW0A/,@corendon.nl,international
18039075278448488,2025-03-16 14:00:14,"Zon, zee en eindeloze siesta’s… ????☀️ In Span...",0,92,https://www.instagram.com/p/DHQ0Zedsckb/,@corendon.nl,international
18085134403543445,2025-03-16 09:00:39,"☀️ Welkom bij Voyage Belek Golf & Spa, waar ee...",1,131,https://www.instagram.com/reel/DHQSGJMB3Nl/,@corendon.nl,international
17842557165447487,2025-03-15 09:00:32,Hotel Diamond Excellence Resort & Spa in Side ...,0,164,https://www.instagram.com/p/DHNtTrRsRIj/,@corendon.nl,international
18046848881361680,2025-03-14 13:01:48,Last-Minute Deal: €70 Korting op het Ladies of...,5,249,https://www.instagram.com/p/DHLkIA1sQxy/,@corendon.nl,international
...,...,...,...,...,...,...,...
17868232276456526,2019-08-23 09:39:08,Heeft u mooie foto’s van uw eigen cruise gemaa...,0,22,https://www.instagram.com/p/B1gHPoyCygq/,@cruisetravelnl,local
17847898231507730,2019-07-15 14:54:46,Onze ambassadeur Henk van der Noort heeft een ...,2,24,https://www.instagram.com/p/Bz8QXReChhs/,@cruisetravelnl,local
18079004770055863,2019-07-01 10:31:44,Onze ambassadeur A. Prevo ging aan boord van d...,0,23,https://www.instagram.com/p/BzXvIioiGI2/,@cruisetravelnl,local


## Feature engineering

### Collaboration indicator 

During the **literature review** the team assumed that the presence of **a tag @** into a post description that points to another account means the fact of collaboration in this particular post

In [327]:
collab_final_df = selected_final_df.copy()
collab_final_df['is_collaboration'] = selected_final_df.apply(
    lambda row: extract_mention_match(row['message'], row['account']),
    axis=1
)

collab_final_df

,date,message,number_of_comments,number_of_likes,link,account,type,is_collaboration
message_id,,,,,,,,
18038388467208858,2025-03-17 19:01:17,??Ontvang nu extra vroegboekkorting tot €200 é...,3,182,https://www.instagram.com/reel/DHT7mD7PW0A/,@corendon.nl,international,False
18039075278448488,2025-03-16 14:00:14,"Zon, zee en eindeloze siesta’s… ????☀️ In Span...",0,92,https://www.instagram.com/p/DHQ0Zedsckb/,@corendon.nl,international,False
18085134403543445,2025-03-16 09:00:39,"☀️ Welkom bij Voyage Belek Golf & Spa, waar ee...",1,131,https://www.instagram.com/reel/DHQSGJMB3Nl/,@corendon.nl,international,False
17842557165447487,2025-03-15 09:00:32,Hotel Diamond Excellence Resort & Spa in Side ...,0,164,https://www.instagram.com/p/DHNtTrRsRIj/,@corendon.nl,international,False
18046848881361680,2025-03-14 13:01:48,Last-Minute Deal: €70 Korting op het Ladies of...,5,249,https://www.instagram.com/p/DHLkIA1sQxy/,@corendon.nl,international,False
...,...,...,...,...,...,...,...,...
17868232276456526,2019-08-23 09:39:08,Heeft u mooie foto’s van uw eigen cruise gemaa...,0,22,https://www.instagram.com/p/B1gHPoyCygq/,@cruisetravelnl,local,False
17847898231507730,2019-07-15 14:54:46,Onze ambassadeur Henk van der Noort heeft een ...,2,24,https://www.instagram.com/p/Bz8QXReChhs/,@cruisetravelnl,local,False
18079004770055863,2019-07-01 10:31:44,Onze ambassadeur A. Prevo ging aan boord van d...,0,23,https://www.instagram.com/p/BzXvIioiGI2/,@cruisetravelnl,local,False


### Day of the week
In the further **day of the week** will be used as an independent variable for statistical testing
The field is produced from date

First of all, cast the `date` column to `pd.datetime`

In [328]:
day_of_the_week_final_df = collab_final_df.copy()
day_of_the_week_final_df['date'] = day_of_the_week_final_df['date'].apply(lambda x: pd.to_datetime(x))

day_of_the_week_final_df.dtypes

date                  datetime64[ns]
message                       object
number_of_comments             int64
number_of_likes               object
link                          object
account                       object
type                          object
is_collaboration                bool
dtype: object

Extract the **day of the week** as a new feature

In [329]:
day_of_the_week_final_df['day_of_the_week'] = day_of_the_week_final_df['date'].dt.day_name()
day_of_the_week_final_df['day_of_the_week'].unique()

array(['Monday', 'Sunday', 'Saturday', 'Friday', 'Thursday', 'Wednesday',
       'Tuesday'], dtype=object)

### Weekend vs weekdays
From `date` field it's also necessary to extract the flag **weekend vs weekends** (Mon-Fri are weekdays, Sat-Sun - weekends) for the further analysis

In [330]:
weekend_vs_weekdays_final_df = day_of_the_week_final_df.copy()
weekend_vs_weekdays_final_df['is_weekend'] = weekend_vs_weekdays_final_df['date'].dt.weekday.apply(
    lambda x: True if x >= 5 else False
)

weekend_vs_weekdays_final_df

,date,message,number_of_comments,number_of_likes,link,account,type,is_collaboration,day_of_the_week,is_weekend
message_id,,,,,,,,,,
18038388467208858,2025-03-17 19:01:17,??Ontvang nu extra vroegboekkorting tot €200 é...,3,182,https://www.instagram.com/reel/DHT7mD7PW0A/,@corendon.nl,international,False,Monday,False
18039075278448488,2025-03-16 14:00:14,"Zon, zee en eindeloze siesta’s… ????☀️ In Span...",0,92,https://www.instagram.com/p/DHQ0Zedsckb/,@corendon.nl,international,False,Sunday,True
18085134403543445,2025-03-16 09:00:39,"☀️ Welkom bij Voyage Belek Golf & Spa, waar ee...",1,131,https://www.instagram.com/reel/DHQSGJMB3Nl/,@corendon.nl,international,False,Sunday,True
17842557165447487,2025-03-15 09:00:32,Hotel Diamond Excellence Resort & Spa in Side ...,0,164,https://www.instagram.com/p/DHNtTrRsRIj/,@corendon.nl,international,False,Saturday,True
18046848881361680,2025-03-14 13:01:48,Last-Minute Deal: €70 Korting op het Ladies of...,5,249,https://www.instagram.com/p/DHLkIA1sQxy/,@corendon.nl,international,False,Friday,False
...,...,...,...,...,...,...,...,...,...,...
17868232276456526,2019-08-23 09:39:08,Heeft u mooie foto’s van uw eigen cruise gemaa...,0,22,https://www.instagram.com/p/B1gHPoyCygq/,@cruisetravelnl,local,False,Friday,False
17847898231507730,2019-07-15 14:54:46,Onze ambassadeur Henk van der Noort heeft een ...,2,24,https://www.instagram.com/p/Bz8QXReChhs/,@cruisetravelnl,local,False,Monday,False
18079004770055863,2019-07-01 10:31:44,Onze ambassadeur A. Prevo ging aan boord van d...,0,23,https://www.instagram.com/p/BzXvIioiGI2/,@cruisetravelnl,local,False,Monday,False


### Personal words
During **literature review** we defined that the presence of personal words in the post description can increase the level of engagement.

Below the list of personal words in **English** and **Dutch** is defined since post of companies we consider contain words in 2 languages

In [331]:
PERSONAL_WORDS = {'you', 'your', 'we', 'us', 'our', 'jou', 'jouw', 'je', 'jij', 'wij', 'ons', 'onze'}

Extract the flag of usage the words above 

In [332]:
personal_words_df = weekend_vs_weekdays_final_df.copy()
personal_words_df['has_personal_words'] = personal_words_df['message'].apply(lambda x: contains_personal_words(x, PERSONAL_WORDS))

personal_words_df

,date,message,number_of_comments,number_of_likes,link,account,type,is_collaboration,day_of_the_week,is_weekend,has_personal_words
message_id,,,,,,,,,,,
18038388467208858,2025-03-17 19:01:17,??Ontvang nu extra vroegboekkorting tot €200 é...,3,182,https://www.instagram.com/reel/DHT7mD7PW0A/,@corendon.nl,international,False,Monday,False,True
18039075278448488,2025-03-16 14:00:14,"Zon, zee en eindeloze siesta’s… ????☀️ In Span...",0,92,https://www.instagram.com/p/DHQ0Zedsckb/,@corendon.nl,international,False,Sunday,True,True
18085134403543445,2025-03-16 09:00:39,"☀️ Welkom bij Voyage Belek Golf & Spa, waar ee...",1,131,https://www.instagram.com/reel/DHQSGJMB3Nl/,@corendon.nl,international,False,Sunday,True,True
17842557165447487,2025-03-15 09:00:32,Hotel Diamond Excellence Resort & Spa in Side ...,0,164,https://www.instagram.com/p/DHNtTrRsRIj/,@corendon.nl,international,False,Saturday,True,True
18046848881361680,2025-03-14 13:01:48,Last-Minute Deal: €70 Korting op het Ladies of...,5,249,https://www.instagram.com/p/DHLkIA1sQxy/,@corendon.nl,international,False,Friday,False,True
...,...,...,...,...,...,...,...,...,...,...,...
17868232276456526,2019-08-23 09:39:08,Heeft u mooie foto’s van uw eigen cruise gemaa...,0,22,https://www.instagram.com/p/B1gHPoyCygq/,@cruisetravelnl,local,False,Friday,False,True
17847898231507730,2019-07-15 14:54:46,Onze ambassadeur Henk van der Noort heeft een ...,2,24,https://www.instagram.com/p/Bz8QXReChhs/,@cruisetravelnl,local,False,Monday,False,True
18079004770055863,2019-07-01 10:31:44,Onze ambassadeur A. Prevo ging aan boord van d...,0,23,https://www.instagram.com/p/BzXvIioiGI2/,@cruisetravelnl,local,False,Monday,False,True


### Length of post
During the **literature research** the team found out that **length of description in words** affects the engagement level.
In the further analysis we'll test two groups:
- posts which length **>= average length of post in words in the account**
- posts which length **< average length of post in words in the account**

For every account the average length of posts is different and it's necessary to calculate

In [333]:
length_post_final_df = personal_words_df.copy()
length_post_final_df['word_count'] = length_post_final_df['message'].apply(lambda x: len(x.split()))

avg_word_counts = length_post_final_df.groupby('account')['word_count'].mean()
avg_word_counts

account
@accessibletravelnetherlands     52.593137
@cherrytraveltours               22.518987
@corendon.nl                     39.001193
@cruisetravelnl                  63.722826
@dreizenvakanties                51.834523
@elizawashere                    72.034335
@hollandtravel                   79.572222
@lvtravelagency                  43.020000
@prijsvrijvakanties              69.609868
@sunweb                          47.179143
@tenzing_travel                 108.919118
@tomstraveltours                 32.785607
@tuinederland                    30.910927
@vakantiediscounter              59.140615
Name: word_count, dtype: float64

Finally, form the new feature and clean the dataset up

In [334]:
length_post_final_df['avg_word_count'] = length_post_final_df['account'].map(avg_word_counts)
length_post_final_df['is_above_avg_length'] = (length_post_final_df['word_count'] > length_post_final_df['avg_word_count'])

# Clean up the dataset
length_post_final_df = length_post_final_df.drop(['word_count', 'avg_word_count'], axis=1)

length_post_final_df

,date,message,number_of_comments,number_of_likes,link,account,type,is_collaboration,day_of_the_week,is_weekend,has_personal_words,is_above_avg_length
message_id,,,,,,,,,,,,
18038388467208858,2025-03-17 19:01:17,??Ontvang nu extra vroegboekkorting tot €200 é...,3,182,https://www.instagram.com/reel/DHT7mD7PW0A/,@corendon.nl,international,False,Monday,False,True,True
18039075278448488,2025-03-16 14:00:14,"Zon, zee en eindeloze siesta’s… ????☀️ In Span...",0,92,https://www.instagram.com/p/DHQ0Zedsckb/,@corendon.nl,international,False,Sunday,True,True,True
18085134403543445,2025-03-16 09:00:39,"☀️ Welkom bij Voyage Belek Golf & Spa, waar ee...",1,131,https://www.instagram.com/reel/DHQSGJMB3Nl/,@corendon.nl,international,False,Sunday,True,True,False
17842557165447487,2025-03-15 09:00:32,Hotel Diamond Excellence Resort & Spa in Side ...,0,164,https://www.instagram.com/p/DHNtTrRsRIj/,@corendon.nl,international,False,Saturday,True,True,True
18046848881361680,2025-03-14 13:01:48,Last-Minute Deal: €70 Korting op het Ladies of...,5,249,https://www.instagram.com/p/DHLkIA1sQxy/,@corendon.nl,international,False,Friday,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...
17868232276456526,2019-08-23 09:39:08,Heeft u mooie foto’s van uw eigen cruise gemaa...,0,22,https://www.instagram.com/p/B1gHPoyCygq/,@cruisetravelnl,local,False,Friday,False,True,False
17847898231507730,2019-07-15 14:54:46,Onze ambassadeur Henk van der Noort heeft een ...,2,24,https://www.instagram.com/p/Bz8QXReChhs/,@cruisetravelnl,local,False,Monday,False,True,False
18079004770055863,2019-07-01 10:31:44,Onze ambassadeur A. Prevo ging aan boord van d...,0,23,https://www.instagram.com/p/BzXvIioiGI2/,@cruisetravelnl,local,False,Monday,False,True,False


### Type of content
During the **literature review** two types of content were defined:
- **photo** 
- **video**

In the given data type of content is defined by `link`:
- link has the same start == `https://instagram.com`
- the following word defines the type of content:
    - `tv` | `reel` - **video**
    - `p` - **photo**
- the rest is `post id`  

Apart from that, some of the rows don't follow the pattern and the content type can't be recognised there (we mark it `unknown`) 

In [335]:
type_of_content_df = length_post_final_df.copy()
type_of_content_df['second_link_element'] = type_of_content_df['link'].str.extract(r'^(?:https?:\/\/)?(?:www\.)?[^\/]+\/([^\/]+)')

type_of_content_df['second_link_element'].unique() 

array(['reel', 'p', 'tv', 'tuinederland'], dtype=object)

As it's possible to see there is one more type - `tuinederland`. We checked it out and find out that this is also `reel`

In [336]:
type_of_content_df[type_of_content_df['second_link_element'] == 'tuinederland']

,date,message,number_of_comments,number_of_likes,link,account,type,is_collaboration,day_of_the_week,is_weekend,has_personal_words,is_above_avg_length,second_link_element
message_id,,,,,,,,,,,,,
18024604451387183,2024-09-30 07:15:46,Hoe je een jetlag kunt voorkomen... en zo meer...,0,116,https://www.instagram.com/tuinederland/reel/DA...,@tuinederland,international,False,Monday,False,True,False,tuinederland


In [337]:
type_of_content_df.loc[
    type_of_content_df['second_link_element'] == 'tuinederland',
    'second_link_element'
] = 'reel'

type_of_content_df['second_link_element'].unique() 

array(['reel', 'p', 'tv'], dtype=object)

Finally, we're creating binary feature `is_photo` and clean up the dataset

In [338]:
type_of_content_df['is_photo'] = type_of_content_df['second_link_element'] == 'p'
type_of_content_df = type_of_content_df.drop(['second_link_element'], axis=1)

type_of_content_df

,date,message,number_of_comments,number_of_likes,link,account,type,is_collaboration,day_of_the_week,is_weekend,has_personal_words,is_above_avg_length,is_photo
message_id,,,,,,,,,,,,,
18038388467208858,2025-03-17 19:01:17,??Ontvang nu extra vroegboekkorting tot €200 é...,3,182,https://www.instagram.com/reel/DHT7mD7PW0A/,@corendon.nl,international,False,Monday,False,True,True,False
18039075278448488,2025-03-16 14:00:14,"Zon, zee en eindeloze siesta’s… ????☀️ In Span...",0,92,https://www.instagram.com/p/DHQ0Zedsckb/,@corendon.nl,international,False,Sunday,True,True,True,True
18085134403543445,2025-03-16 09:00:39,"☀️ Welkom bij Voyage Belek Golf & Spa, waar ee...",1,131,https://www.instagram.com/reel/DHQSGJMB3Nl/,@corendon.nl,international,False,Sunday,True,True,False,False
17842557165447487,2025-03-15 09:00:32,Hotel Diamond Excellence Resort & Spa in Side ...,0,164,https://www.instagram.com/p/DHNtTrRsRIj/,@corendon.nl,international,False,Saturday,True,True,True,True
18046848881361680,2025-03-14 13:01:48,Last-Minute Deal: €70 Korting op het Ladies of...,5,249,https://www.instagram.com/p/DHLkIA1sQxy/,@corendon.nl,international,False,Friday,False,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17868232276456526,2019-08-23 09:39:08,Heeft u mooie foto’s van uw eigen cruise gemaa...,0,22,https://www.instagram.com/p/B1gHPoyCygq/,@cruisetravelnl,local,False,Friday,False,True,False,True
17847898231507730,2019-07-15 14:54:46,Onze ambassadeur Henk van der Noort heeft een ...,2,24,https://www.instagram.com/p/Bz8QXReChhs/,@cruisetravelnl,local,False,Monday,False,True,False,True
18079004770055863,2019-07-01 10:31:44,Onze ambassadeur A. Prevo ging aan boord van d...,0,23,https://www.instagram.com/p/BzXvIioiGI2/,@cruisetravelnl,local,False,Monday,False,True,False,True


### Peak vs non-peak
According to **literature review** posts in peak season (it's June, July, August and December) have bigger engagement rates.

In [343]:
peak_final_df = type_of_content_df.copy()
peak_final_df['is_peak'] = peak_final_df['date'].dt.month.isin([6, 7, 8, 12]).astype(bool)

peak_final_df

,date,message,number_of_comments,number_of_likes,link,account,type,is_collaboration,day_of_the_week,is_weekend,has_personal_words,is_above_avg_length,is_photo,is_peak
message_id,,,,,,,,,,,,,,
18038388467208858,2025-03-17 19:01:17,??Ontvang nu extra vroegboekkorting tot €200 é...,3,182,https://www.instagram.com/reel/DHT7mD7PW0A/,@corendon.nl,international,False,Monday,False,True,True,False,False
18039075278448488,2025-03-16 14:00:14,"Zon, zee en eindeloze siesta’s… ????☀️ In Span...",0,92,https://www.instagram.com/p/DHQ0Zedsckb/,@corendon.nl,international,False,Sunday,True,True,True,True,False
18085134403543445,2025-03-16 09:00:39,"☀️ Welkom bij Voyage Belek Golf & Spa, waar ee...",1,131,https://www.instagram.com/reel/DHQSGJMB3Nl/,@corendon.nl,international,False,Sunday,True,True,False,False,False
17842557165447487,2025-03-15 09:00:32,Hotel Diamond Excellence Resort & Spa in Side ...,0,164,https://www.instagram.com/p/DHNtTrRsRIj/,@corendon.nl,international,False,Saturday,True,True,True,True,False
18046848881361680,2025-03-14 13:01:48,Last-Minute Deal: €70 Korting op het Ladies of...,5,249,https://www.instagram.com/p/DHLkIA1sQxy/,@corendon.nl,international,False,Friday,False,True,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17868232276456526,2019-08-23 09:39:08,Heeft u mooie foto’s van uw eigen cruise gemaa...,0,22,https://www.instagram.com/p/B1gHPoyCygq/,@cruisetravelnl,local,False,Friday,False,True,False,True,True
17847898231507730,2019-07-15 14:54:46,Onze ambassadeur Henk van der Noort heeft een ...,2,24,https://www.instagram.com/p/Bz8QXReChhs/,@cruisetravelnl,local,False,Monday,False,True,False,True,True
18079004770055863,2019-07-01 10:31:44,Onze ambassadeur A. Prevo ging aan boord van d...,0,23,https://www.instagram.com/p/BzXvIioiGI2/,@cruisetravelnl,local,False,Monday,False,True,False,True,True


### Used features dropping

Used features that won't be used in the further analysis and therefore must be dropped:

- `message`
- `date`
- `link`

In [339]:
USED_FEATURES_TO_DROP = [
    'message',
    'date',
    'link'
]

Form the final dataset for the analysis (still with outliers) and save it

In [345]:
final_new_df = peak_final_df.drop(USED_FEATURES_TO_DROP, axis=1)
final_new_df.to_excel('./final_with_new_features.xlsx', index=False)
final_new_df

,number_of_comments,number_of_likes,account,type,is_collaboration,day_of_the_week,is_weekend,has_personal_words,is_above_avg_length,is_photo,is_peak
message_id,,,,,,,,,,,
18038388467208858,3,182,@corendon.nl,international,False,Monday,False,True,True,False,False
18039075278448488,0,92,@corendon.nl,international,False,Sunday,True,True,True,True,False
18085134403543445,1,131,@corendon.nl,international,False,Sunday,True,True,False,False,False
17842557165447487,0,164,@corendon.nl,international,False,Saturday,True,True,True,True,False
18046848881361680,5,249,@corendon.nl,international,False,Friday,False,True,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...
17868232276456526,0,22,@cruisetravelnl,local,False,Friday,False,True,False,True,True
17847898231507730,2,24,@cruisetravelnl,local,False,Monday,False,True,False,True,True
18079004770055863,0,23,@cruisetravelnl,local,False,Monday,False,True,False,True,True


## EDA for final dataset

## Outliers removal

# Statistical testing

# Multi-level regression analysis